## Project Description

Imagine you're developing a deep learning system tailored for sentiment analysis of product reviews, specifically for a newly established online beautiy product retail company. The goal is to assist the company in making informed decisions about inventory management – deciding what products to retain and what to remove from stock. The company, keen on enhancing customer satisfaction, has been actively monitoring comments on their website and has invested in annotators to label sentiments. They hand you a dataset comprising 80,000 customer reviews, each labeled with 0 for negative sentiment and 1 for positive sentiment. After extensive effort and refinement, you successfully train and deploy a classifier that predicts sentiment based on online comments. Excitedly, you report an 86% accuracy on a held-out test set to your bosses. However, to your disappointment, management expresses dissatisfaction, insisting on a minimum of 90% accuracy before considering the widespread implementation of the AI model. 
You suspect that certain annotators might have made errors, potentially affecting your model's effectiveness. Empowered by a newfound "confidence," you opt for "confidence" learning to pinpoint and rectify any inaccuracies in the dataset before embarking on the retraining process once more.

First, we prepare the environment for AWS SageMaker operations by setting up clients and retrieving essential configuration details like the default S3 bucket, execution role, and AWS region. 

In [1]:
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
import botocore
from botocore.exceptions import ClientError

config = botocore.config.Config(user_agent_extra='dlai-pds/c2/w3')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


We then configure the data source for a training job in SageMaker, defining where the training data is located (in this case, an S3 bucket) and the nature of the data.

In [2]:
from sagemaker.inputs import TrainingInput

# TODO: set the path to the train data
train_data = TrainingInput(
    's3://rajat369007bucket/data/', 
    content_type='application/x-sagemaker-training-data'
)


A PyTorch estimator with the specified configurations for a SageMaker training job is created. The training job will use the provided entry point script, run on the specified instance type, and output the trained model to the specified S3 path. The entry point script main.py contains the main steps that needs to be completed in this project.

In [3]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point= "main.py",
    source_dir= "./",
    base_job_name="sagemaker-script-mode",
    role=role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    framework_version="2.1",
    py_version="py310",
    dependencies= None,
    output_data_config={
        'S3OutputPath': 's3://rajat369007bucket/output/'
    },
    output_path= "s3://sagemaker-us-east-1-456165959068/output/",
    environment={'PYTHONPATH': 'src'}
)

The following script sets up a ModelCheckpoint callback to automatically save the best model (based on development loss) during the training process in a SageMaker training job. The best model will be stored at the specified directory path within the SageMaker environment.

In [4]:
# Save the best model during training by specifying the output path
# (Note: The output path should be where the best model will be saved within the S3 bucket)
model_checkpoint = {
    'ModelCheckpoint': {
        'monitor': 'dev_loss',
        'dirpath': '/opt/ml/model/',
        'filename': 'best_model',
        'save_top_k': 1,
        'mode': 'min'
    }
}

# Attach the ModelCheckpoint callback to the estimator
estimator._hyperparameters['callbacks'] = [model_checkpoint]


Starting the training process: 

In [5]:
estimator.fit({"train": train_data})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: sagemaker-script-mode-2024-05-07-20-23-36-142


2024-05-07 20:23:39 Starting - Starting the training job...
2024-05-07 20:23:40 Pending - Training job waiting for capacity............
2024-05-07 20:25:45 Pending - Preparing the instances for training...
2024-05-07 20:26:37 Downloading - Downloading the training image.....................
2024-05-07 20:29:49 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-05-07 20:30:08,864 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-05-07 20:30:08,882 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-07 20:30:08,894 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-05-07 20:30:08,896 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-05-07 20:30:11,421 sagemaker-training-toolkit INFO     Instal

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: /opt/ml/code/lightning_logs
| Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 49.3 K
-------------------------------------
49.3 K    Trainable params
0         Non-trainable params
49.3 K    Total params
0.197     Total estimated model params size (MB)
Sanity Checking: 0it [00:00, ?it/s]
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Epoch 0:  80%|████████  | 1600/2000 [00:11<00:02, 134.76it/s, loss=0.232, v_num=0, train_loss=0.137, train_acc=0.938]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2000/2000 [00:13<00:00, 146.28it/s, loss=0.232, v_num=0, train_loss=0.137, train_acc=0.938, dev_loss=0.241, dev_acc=0.903]
Epoch 0, global step 1600: 'dev_loss' reached 0.24149 (best 0.24149), saving model to './log/epoch=0-step=1600.ckpt' as top 1
Epoch 0: 100%|██████████| 2000/2000 [00:13<00:00, 146.22it/s, loss=0.232, v_num=0, train_loss=0.137, train_acc=0.938, dev_loss=0.241, dev_acc=0.903]
Restoring states from the checkpoint path at ./log/epoch=0-step=1600.ckpt
Loaded model weights from checkpoint at ./log/epoch=0-step=1600.ckpt
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which 

Testing DataLoader 0: 100%|██████████| 500/500 [00:02<00:00, 237.13it/s]
{'acc': 0.9081249833106995, 'loss': 0.23860545456409454}
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory ./log exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dir

...

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
| Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 49.3 K
-------------------------------------
49.3 K    Trainable params
0         Non-trainable params
49.3 K    Total params
0.197     Total estimated model params size (MB)
Sanity Checking: 0it [00:00, ?it/s]
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine)


2024-05-07 20:31:44 Uploading - Uploading generated training modelEpoch 0:  30%|███       | 603/2000 [00:04<00:10, 132.59it/s, loss=0.692, v_num=4, train_loss=0.681, train_acc=0.594]
Epoch 0:  54%|█████▍    | 1088/2000 [00:08<00:06, 132.27it/s, loss=0.694, v_num=4, train_loss=0.697, train_acc=0.375]


Epoch 0:  80%|████████  | 1600/2000 [00:12<00:03, 132.11it/s, loss=0.696, v_num=4, train_loss=0.694, train_acc=0.406]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2000/2000 [00:13<00:00, 143.25it/s, loss=0.696, v_num=4, train_loss=0.694, train_acc=0.406, dev_loss=0.693, dev_acc=0.499]
Restoring states from the checkpoint path at /opt/ml/model/best_model.ckpt
Loaded model weights from checkpoint at /opt/ml/model/best_model.ckpt
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Testing: 0it [00:00, ?it/s]
Testing DataLoader 0:  33%|███▎      | 167/500 [00:00<00:01, 228.82it/s]


Testing DataLoader 0: 100%|██████████| 500/500 [00:02<00:00, 231.41it/s]
{'acc': 0.49262499809265137, 'loss': 0.6935612559318542}
done! great work!
2024-05-07 20:31:41,615 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2024-05-07 20:31:41,615 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2024-05-07 20:31:41,615 sagemaker-training-toolkit INFO     Reporting training SUCCESS



2024-05-07 20:32:00 Completed - Training job completed
Training seconds: 343
Billable seconds: 343


## Model Deployment

We need to copy the training artifacts, i.e, output.tar.gz, from the corresponding S3 bucket to the current working directory.

In [8]:
# copy the training artifacts from the S3 bucket to the current working directory
!aws s3 cp "s3://sagemaker-us-east-1-456165959068/output/sagemaker-script-mode-2024-05-07-20-23-36-142/output/output.tar.gz" .     

download: s3://sagemaker-us-east-1-456165959068/output/sagemaker-script-mode-2024-05-07-20-23-36-142/output/output.tar.gz to ./output.tar.gz


We can decompress the training artifacts to `extracted_files` for further exploration.

In [9]:
!tar -xzf output.tar.gz -C extracted_training_artifacts

tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


We then create an endpoint 'sentiment-analysis-endpoint-2' and deploy the model to that endpoint.

In [12]:
# deploy the trained model
predictor = estimator.deploy(instance_type="ml.p3.2xlarge", initial_instance_count=1)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-456165959068/output/sagemaker-script-mode-2024-05-07-20-23-36-142/output/model.tar.gz), script artifact (s3://sagemaker-us-east-1-456165959068/sagemaker-script-mode-2024-05-07-20-23-36-142/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-456165959068/sagemaker-script-mode-2024-05-07-20-46-45-230/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: sagemaker-script-mode-2024-05-07-20-46-45-230
INFO:sagemaker:Creating endpoint-config with name sagemaker-script-mode-2024-05-07-20-46-45-230
INFO:sagemaker:Creating endpoint with name sagemaker-script-mode-2024-05-07-20-46-45-230


-----------!